In [2]:
import numpy as np
import sentence_metrix as sm

In [88]:
embed_word_metrix = np.load('./data/embeddings.npy')

In [89]:
embed_word_metrix.shape

(21161, 20)

In [90]:
vocab = np.load('./data/vocab.pkl')

In [91]:
len(vocab)

21161

In [92]:
embed_word_metrix[0,:].shape

(20,)

In [93]:
	sentence_metrix = []
	for fn in ["./twitter-datasets/pos_train.txt","./twitter-datasets/neg_train.txt"]:
			with open(fn) as f:
				for line in f:
					for t in line.strip().split():
						sentence_vector = np.zeros(20) 
						if(vocab.get(t)!=None):
							sentence_vector +=embed_word_metrix[vocab.get(t)-1,:]
					sentence_metrix.append(sentence_vector)

In [94]:
sentence_metrix = np.asarray(sentence_metrix)

In [95]:
sentence_metrix.shape

(200000, 20)

In [96]:
sentence_metrix

array([[-0.58216098, -0.05721694, -0.24918398, ..., -0.23684238,
        -0.60419656, -0.56451865],
       [-0.20708859, -0.09317281, -0.27406503, ..., -0.09272619,
        -0.0238416 ,  0.16086663],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [-0.20708859, -0.09317281, -0.27406503, ..., -0.09272619,
        -0.0238416 ,  0.16086663],
       [-0.64775718, -0.24338572, -0.71112321, ..., -0.14152039,
        -0.02763016,  0.44240767],
       [-0.67838218, -0.15581983, -0.17458961, ..., -0.60721204,
         0.26350759,  0.61909953]])

In [98]:
y_train = np.zeros(200000)
for i in range(0,100000,1):
    y_train[i]=1
for i in range(100000,200000,1):
    y_train[i] = -1

In [99]:
from sklearn import linear_model

In [100]:
reg = linear_model.LinearRegression()
reg.fit(sentence_metrix,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [101]:
reg.coef_

array([-0.08604567, -0.04366987, -0.08893452,  0.0044943 ,  0.01917628,
       -0.08401141,  0.05529616,  0.02881053, -0.02830876,  0.01909553,
       -0.05358102, -0.03211675,  0.04232069, -0.04246669,  0.02455039,
       -0.0140496 ,  0.053654  ,  0.00684581, -0.00956578,  0.04576188])

In [102]:
reg.intercept_

-0.20329452831123362

In [103]:
w = np.insert(reg.coef_,0,reg.intercept_)

In [104]:
w.shape

(21,)

In [105]:
	sentence_metrix_test = []
	for fn in ["./twitter-datasets/test_data.txt"]:
			with open(fn) as f:
				for line in f:
					for t in line.strip().split():
						sentence_vector = np.zeros(20) 
						if(vocab.get(t)!=None):
							sentence_vector +=embed_word_metrix[vocab.get(t)-1,:]
					sentence_metrix_test.append(sentence_vector)

In [106]:
sentence_metrix_test = np.asarray(sentence_metrix_test)

In [107]:
sentence_metrix_test.shape

(10000, 20)

In [108]:
sentence_metrix_test = np.insert(sentence_metrix_test,0,1,axis=1)

In [109]:
test_y = sentence_metrix_test.dot(w)
test_y

array([-0.07255184,  0.0077238 ,  0.082897  , ..., -0.07255184,
       -0.20329453, -0.07255184])

In [110]:
for y_index in test_y:
    if (y_index>0):
        y_index = 1
    else:
        y_index = -1
        

In [111]:
y_label = np.zeros(test_y.shape)

In [112]:
for i in range(0,10000,1):
    if (test_y[i]>0):
        y_label[i]  = 1
    else:
        y_label[i] = -1

In [113]:
y_label

array([-1.,  1.,  1., ..., -1., -1., -1.])

In [114]:
from helpers import create_csv_submission

In [115]:
OUTPUT_PATH = './data/first.csv' 
create_csv_submission(range(1,10001,1), y_label, OUTPUT_PATH)
#first try using least square (59.320%)